# Time series using STAC API statistics endpoints

This notebook demonstrates how to generate a timeseries using STAC API statistics endpoints.

* Author: Leo Thomas
* Lasted Updated Date: June 29, 2022

In [1]:
import concurrent.futures
import datetime as dt
from ipyleaflet import basemaps, Map, GeoJSON
import json
import requests as re
import matplotlib.pyplot as plt
import pprint
import time

STAC_API_URL = "https://staging-stac.delta-backend.xyz"
RASTER_API_URL = "https://staging-raster.delta-backend.xyz"

# Declare your collection of interest

You can discover available collections the following ways:

* Use the `{STAC_ENDPOINT_URL}/collections` API endpoint (JSON response)
* Programmatically using `pystac` (see example in the `list-collections.ipynb` notebook
* In the STAC Browser: http://delta-staging-stac-browser.s3-website-us-east-1.amazonaws.com/


In [2]:
collection = 'no2-monthly'

## Discover data using the STAC endpoint

In [3]:
re.get(f"{STAC_API_URL}/collections/{collection}").json()

{'id': 'no2-monthly',
 'type': 'Collection',
 'links': [{'rel': 'items',
   'type': 'application/geo+json',
   'href': 'https://staging-stac.delta-backend.xyz/collections/no2-monthly/items'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://staging-stac.delta-backend.xyz/'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://staging-stac.delta-backend.xyz/'},
  {'rel': 'self',
   'type': 'application/json',
   'href': 'https://staging-stac.delta-backend.xyz/collections/no2-monthly'}],
 'title': 'NO₂',
 'extent': {'spatial': {'bbox': [[-180, -90, 180, 90]]},
  'temporal': {'interval': [['2016-01-01T00:00:00Z',
     '2022-01-01T00:00:00Z']]}},
 'license': 'MIT',
 'summaries': {'datetime': ['2016-01-01T00:00:00Z', '2022-05-01T00:00:00Z'],
  'cog_default': {'max': 50064805976866816, 'min': -6618294421291008}},
 'description': 'Darker colors indicate higher nitrogen dioxide (NO₂) levels and more activity. Lighter colors indicate lower levels of NO₂ an

## Describe the periodic nature of the data

In [4]:
pprint.pprint({
    k:v for k,v in re.get(f"{STAC_ENDPOINT_URL}/collections/no2-monthly").json().items()
    if k in ["dashboard:is_periodic", "dashboard:time_density", "summaries"]
})

NameError: name 'STAC_ENDPOINT_URL' is not defined

## Load and inspect one of the STAC items

This step is just for demonstration, to inspect what an item looks like.

In [ ]:
items = re.get(f"{STAC_ENDPOINT_URL}/collections/{collection}/items?limit=100").json()["features"]
items[0]

## Define a bounding box

We've defined a bounding box for France in this case.

In [ ]:
bounding_box_france = { 
    "type": "Feature",
    "properties": {},
    "geometry": {
        "type": "Polygon",
        "coordinates": [[
            [
              -5.4534286,
              41.2632185
            ],
            [
              9.8678344,
              41.2632185
            ],
            [
              9.8678344,
              51.268318
            ],
            [
              -5.4534286,
              51.268318
            ],
            [
              -5.4534286,
              41.2632185
            ]
        ]]
    }
}



## Map the bounding box

This step is for visual inspection of the bounding box.

In [ ]:
m = Map(
    basemap=basemaps.OpenStreetMap.Mapnik,
    center=(47,4),
    zoom=3
)
geo = GeoJSON(
    data=bounding_box_france,
    style={"color": "red", "fillOpacity": 0}
)
m.add_layer(geo)
m

## Use `/cog/statistics` to get data for the bounding box

First, we create a `generate_stats` function and then we call it with the bounding box defined for France.

In [ ]:
def generate_stats(items, bounding_box):
    stats = [
        {
            **re.post(
                f"{RASTER_API_URL}/cog/statistics", 
                params={
                    "url":item["assets"]["cog_default"]["href"]
                },
                json=bounding_box
            ).json()["properties"], 
             "start_datetime":item["properties"]["start_datetime"]
        }
        for item in items
    ]
    return stats

## Generate and estimate time to generate statistics

This may take a minute, depending on the network.

In [ ]:
start = time.time()
stats = generate_stats(items, bounding_box_france)
end = time.time()
print(f"Elapsed time for small bounding box (france) {round(end-start,2)} seconds. Items queried: {len(items)}")

## Inspect one result

In [ ]:
stats[1]

In [ ]:
dates = [dt.datetime.strptime(stat["start_datetime"], "%Y-%m-%dT%H:%M:%SZ") for stat in stats]
means = [stat["statistics"]["1"]["mean"] for stat in stats]
std_devs = [stat["statistics"]["1"]["std"] for stat in stats]
upper_bounds = [m+s for (m,s) in zip(means, std_devs)]
lower_bounds = [m-s for (m,s) in zip(means, std_devs)]

In [ ]:
plt.plot(dates, means, 'black')
plt.fill_between(dates, upper_bounds, lower_bounds, facecolor="lightgrey", interpolate=True)